(1) Import libraries

In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from keras.models import Sequential, Model, load_model
from keras import backend as BK
import cv2
from scipy.io.wavfile import read
from scipy.io.wavfile import write
import scipy.io as sio
from collections import OrderedDict
import random 
random.seed(100)
import fnmatch
import os

Using TensorFlow backend.
C:\Users\sally\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sally\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sally\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sally\AppData\Roa

(2) Define variables and parameters

In [31]:
speaker = 's23'

frame_width =128
frame_height =128
num_frame = 75
slice_length =5
num_slices=15
print(num_slices)

CNN_path = 'D:/Mrs_backup/speech_test/cnn/face/'+ speaker + '/'
face_doc = 'D:/Mrs_backup/speech_test/cnn/face/'+ speaker + ''
audio_path = 'D:/Mrs_backup/speech_test/AudSpecs/'+ speaker+ ''
autoencoder_path= 'D:/Mrs_backup/speech_test/autoencoder/'+ speaker + '_trainModel/'
integrate_data_path ='D:/Mrs_backup/speech_test/cnn/input/'+ speaker + '/'
if not os.path.exists(integrate_data_path):
    os.mkdir(integrate_data_path)
    
print('CNN input path is '+ CNN_path)
print('face_doc path is '+ face_doc)
print('audio path is '+ audio_path)
print('autoencoder path is: '+autoencoder_path)
print('integrate path is: '+integrate_data_path)


15
CNN input path is D:/Mrs_backup/speech_test/cnn/face/s23/
face_doc path is D:/Mrs_backup/speech_test/cnn/face/s23
audio path is D:/Mrs_backup/speech_test/AudSpecs/s23
autoencoder path is: D:/Mrs_backup/speech_test/autoencoder/s23_trainModel/
integrate path is: D:/Mrs_backup/speech_test/cnn/input/s23/


(3) Define functions for video input

In [32]:
# Define load_video_3D function and Cap frames (75) frame width and frame height (128)
def load_video_3D(path):
    cap = cv2.VideoCapture(path)
    frameCount = num_frame
    frameHeight=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT ))
    frameWidth=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH ))
    
    buf =np.empty((frameHeight, frameWidth, frameCount), np.dtype('float32'))
    fc = 0
    ret = True
    
    while (fc < frameCount  and ret):
        ret, frame = cap.read()
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame=frame.astype('float32')
        frame = frame-np.amin(frame)
        frame = frame/np.amax(frame)
        buf[:,:,fc]=frame
        fc += 1
    cap.release()
    return buf

# Define diff function to add 3 time-derivative channels
def diff(buf_input):
    buf_input=np.pad(buf_input,((0,0),(0,0),(1,0)),'edge')
    buf_output=np.diff(buf_input,axis=2)
    return buf_output

# Define slice_video_3D function to divide into 15 non-overlap slices each of length 5
def slice_video_3D(video):
    video_output =np.empty((num_slices,3,frame_height,frame_width,slice_length), np.dtype('float32'))
    
    start=0
    for i in range(0,num_slices):
        video_output[i,:,:,:,:]=video[:,:,:,start:start+slice_length]
        start+=slice_length
    return video_output

In [33]:
j=0
for filename in os.listdir(CNN_path): 
    src =CNN_path+ filename 
    print(src)
    format_num1="{number:03}".format(number=j)
    dst =CNN_path + str(format_num1)+'.avi'
    print(dst)
    os.rename(src, dst)
    j=j+1  

D:/Mrs_backup/speech_test/cnn/face/s23/bbad1s.avi
D:/Mrs_backup/speech_test/cnn/face/s23/000.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bbad2p.avi
D:/Mrs_backup/speech_test/cnn/face/s23/001.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bbad3a.avi
D:/Mrs_backup/speech_test/cnn/face/s23/002.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bbadzn.avi
D:/Mrs_backup/speech_test/cnn/face/s23/003.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bbaj4n.avi
D:/Mrs_backup/speech_test/cnn/face/s23/004.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bbaj5s.avi
D:/Mrs_backup/speech_test/cnn/face/s23/005.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bbaj6p.avi
D:/Mrs_backup/speech_test/cnn/face/s23/006.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bbaj7a.avi
D:/Mrs_backup/speech_test/cnn/face/s23/007.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bbap8n.avi
D:/Mrs_backup/speech_test/cnn/face/s23/008.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bbap9s.avi
D:/Mrs_backup/speech_test/cnn/face/s23/009.avi
D:/Mrs_backup/speech_test/cnn/

D:/Mrs_backup/speech_test/cnn/face/s23/bgie4n.avi
D:/Mrs_backup/speech_test/cnn/face/s23/096.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bgie5s.avi
D:/Mrs_backup/speech_test/cnn/face/s23/097.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bgie6p.avi
D:/Mrs_backup/speech_test/cnn/face/s23/098.avi
D:/Mrs_backup/speech_test/cnn/face/s23/bgie7a.avi
D:/Mrs_backup/speech_test/cnn/face/s23/099.avi


In [34]:
#Create a text file for visual input path
file = open(face_doc +'_valid_videos.txt','w')   
file.close()

# Get the amount of files in face folder 
numfiles=len(fnmatch.filter(os.listdir(CNN_path), '*.avi'))
print(numfiles)

for j in range(0,numfiles):
    format_num="{number:03}".format(number=j)
   
    with open(face_doc +'_valid_videos.txt', 'a') as file:
        file.write(CNN_path+str(format_num)+'.avi\n')

100


(4) Video frame extraction

In [35]:
# Open the text file of auditory spectrogram input path 
text_file = open(face_doc +'_valid_videos.txt', 'r')

# Load each line of the text file
lines = text_file.read().split('\n')
index_shuf=list(range(len(lines)))
lines_shuf=[]
for i in index_shuf:
    lines_shuf.append(lines[i])
    
#Save a dictionary of names and arrays into a MATLAB-style .mat file
sio.savemat('index_s1_2_4_29.mat', mdict={'lines_shuf':lines_shuf}) 
#print(lines_shuf)

# Get the number of videos
num_videos=len(lines)

# Define the shape of the video_input [10*15,3,128,128,5]
video_input =np.empty((num_videos*(num_slices),3,int(frame_height),int(frame_width),int(slice_length)), np.dtype('float32'))

speaker_id=OrderedDict()

# Get video data
i=0
for row in lines_shuf:
    #print(i)
    this_id=row.split('/')[6]
    if this_id in speaker_id:
        speaker_id[this_id]+=1
    else:
        speaker_id[this_id]=1
    
    # Call load_video_3D function to cap frames (75) frame width and frame height (128) from the path
    tmp0=load_video_3D(row)
    
    # Call diff function to add 3 time-derivative channels
    diff_video=np.empty((3,tmp0.shape[0],tmp0.shape[1],tmp0.shape[2]))
    diff_video[0,:,:,:]=tmp0
    diff_video[1,:,:,:]=diff(tmp0)
    diff_video[2,:,:,:]=diff(diff_video[1,:,:,:])
    
    #print(diff_video.shape)
    # Call slice_video_3D function
    data_vid=slice_video_3D(diff_video)
    
    # Add total number of slices 
    video_input[i*(num_slices):(i+1)*(num_slices),:,:,:,:]=data_vid[:,:,:,:,:]
    
    i+=1
    print(i)
    if i>=num_videos:
        break
    if i%10==0:
        print(str(i)+'/'+str(num_videos))
print('Video slices shape:'+str(video_input.shape))

1
2
3
4
5
6
7
8
9
10
10/100
11
12
13
14
15
16
17
18
19
20
20/100
21
22
23
24
25
26
27
28
29
30
30/100
31
32
33
34
35
36
37
38
39
40
40/100
41
42
43
44
45
46
47
48
49
50
50/100
51
52
53
54
55
56
57
58
59
60
60/100
61
62
63
64
65
66
67
68
69
70
70/100
71
72
73
74
75
76
77
78
79
80
80/100
81
82
83
84
85
86
87
88
89
90
90/100
91
92
93
94
95
96
97
98
99
100
Video slices shape:(1500, 3, 128, 128, 5)


(5) Define functions for audio data

In [36]:
# Define slice_audio_spec function to to divide into 15 non-overlap slices each of length 26
def slice_audio_spec(audio_spec):
    global AUDIO_LENGTH
    window_size=int(AUDIO_LENGTH/num_slices) #from time to number of audio index 
    #print(window_size)
    audio_output =np.empty((num_slices,audio_spec.shape[0],window_size), np.dtype('float32'))
    start=0
    for i in range(0,num_slices):
        audio_output[i,:,:]=audio_spec[:,start:start+window_size]
        start+=window_size
        if start>AUDIO_LENGTH-window_size:
            break
    #print(audio_output.shape)
    #print(audio_output[1,:,1])
    return audio_output

# Define get activations function # Extract the 32-bin bottleneck features as target for the main network
def get_activations(model, layer_in, layer_out, X_batch):
    get_activations = BK.function([model.layers[layer_in].input, BK.learning_phase()], [model.layers[layer_out].output])
    activations = get_activations([X_batch,0])
    return activations

#Define padding function
def get_padded_spec(data):
    
    # Compress the spectrogram by raising to the power 1/3
    #print(data[1:3,1])
    data=np.power(data,.3)
    #print(data[1,370])
    
    # Get the video length
    t=data.shape[1]
   
    # Get the number of pads
    num_pads=int((2*num_slices)-(t%(2*num_slices)))
    #print(num_pads)
    
    # Add padding to the video length
    padded_data=np.pad(data,((0,0),(0,num_pads)),'reflect')
    #print(padded_data.shape)

    return padded_data

(6) Load autoencoder model

In [37]:
# Define cost function [mean squared error + correlation loss]
def corr2_mse_loss(a,b):
    a = BK.tf.subtract(a, BK.tf.reduce_mean(a))
    b = BK.tf.subtract(b, BK.tf.reduce_mean(b))
    tmp1 = BK.tf.reduce_sum(BK.tf.multiply(a,a))
    tmp2 = BK.tf.reduce_sum(BK.tf.multiply(b,b))
    tmp3 = BK.tf.sqrt(BK.tf.multiply(tmp1,tmp2))
    tmp4 = BK.tf.reduce_sum(BK.tf.multiply(a,b))
    r = -BK.tf.divide(tmp4,tmp3)
    m=BK.tf.reduce_mean(BK.tf.square(BK.tf.subtract(a, b)))
    rm=BK.tf.add(r,m)
    return rm

# Load autoencoder model
print('Loading autoencoder model...')
config = BK.tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = BK.tf.Session(config=config)
model=load_model(autoencoder_path+'autoencoder.h5',custom_objects={'corr2_mse_loss': corr2_mse_loss})
model.load_weights(autoencoder_path+'autoencoder_weights.h5')

Loading autoencoder model...


In [38]:
#Create a text file for visual input path
file = open(audio_path +'_valid_aud_specs.txt','w')   
file.close()

# Get the amount of files in face folder 
numfiles=len(fnmatch.filter(os.listdir(audio_path+'/'), '*.mat'))
print(numfiles)

for j in range(0,numfiles):
    format_num="{number:03}".format(number=j)
   
    with open(audio_path +'_valid_aud_specs.txt', 'a') as file:
        file.write(audio_path+'/'+str(format_num)+'.mat\n')

100


(7)Audio feature extraction 

In [39]:
# Open the text file of auditory spectrogram input path 
text_file = open(audio_path +'_valid_aud_specs.txt','r')

lines = text_file.read().split('\n')

lines_shuf=[]
for i in index_shuf:
    lines_shuf.append(lines[i])

# Get the number of audios
num_audios=len(lines)

# Get data shape 
#print(lines[0])
mat=sio.loadmat(lines[0])
data = mat['y'].T[:,2:]

# get_padded_spec function to get the shape after padding data
padded_data=get_padded_spec(data=data)
global AUDIO_LENGTH
AUDIO_LENGTH=padded_data.shape[1]

# Call get_activations function to get the shape after bottleneck features
bottleneck=get_activations(model, 0, 12, padded_data.T)
bottleneck=bottleneck[0].T

# Get the total shape of audio_input variable
audio_input =np.empty((num_audios*(num_slices),bottleneck.shape[0],int(AUDIO_LENGTH/num_slices)), np.dtype('float32'))
print("Audio slices shape:" + str(audio_input.shape))

tmp =np.zeros((AUDIO_LENGTH), np.dtype('float32'))


i=0
for row in lines_shuf:
    
    # Load data from the path
    mat=sio.loadmat(row)
    
    # Read data from the second feature
    data = mat['y'].T[:,2:]
    #print(data[:,1])
    
    # Call get_padded_spec function: 
    # (1). Compress the spectrogram by raising to the power 1/3 
    # (2). Add padding to the video length 
    padded_data=get_padded_spec(data=data)
   
    
    # Call get_activations function to get the bottleneck feature from autoencoder model 
    # Encoder auditory spectrogram
    #print(padded_data.T.shape)
    #print(padded_data.T[1,1:10])
    bottleneck=get_activations(model, 0, 12, padded_data.T)
    #print(bottleneck[0].shape)
    #print(bottleneck[0][1,:])
    
    #Transpose bottleneck[0] varaible
    bottleneck=bottleneck[0].T
       
    # Call slice_audio_spec function to divide into 15 non-overlap audio slices each of length [390/15] 26
    data=slice_audio_spec(bottleneck)
    #print(data.shape)
    
    #Get the total audio slices
    audio_input[i*(num_slices):(i+1)*(num_slices),:,:]=data[:,:,:]
    i+=1
    if i>=num_audios:
        break
    if i%10==0:
        print(str(i)+'/'+str(num_audios))

audio_output=np.reshape(audio_input,(audio_input.shape[0],audio_input.shape[1]*audio_input.shape[2]))
print('Audio slices shape:'+str(audio_input.shape))
print('Target features to network shape:'+str(audio_output.shape))

print(speaker_id)

Audio slices shape:(1500, 32, 26)
10/100
20/100
30/100
40/100
50/100
60/100
70/100
80/100
90/100
Audio slices shape:(1500, 32, 26)
Target features to network shape:(1500, 832)
OrderedDict([('000.avi', 1), ('001.avi', 1), ('002.avi', 1), ('003.avi', 1), ('004.avi', 1), ('005.avi', 1), ('006.avi', 1), ('007.avi', 1), ('008.avi', 1), ('009.avi', 1), ('010.avi', 1), ('011.avi', 1), ('012.avi', 1), ('013.avi', 1), ('014.avi', 1), ('015.avi', 1), ('016.avi', 1), ('017.avi', 1), ('018.avi', 1), ('019.avi', 1), ('020.avi', 1), ('021.avi', 1), ('022.avi', 1), ('023.avi', 1), ('024.avi', 1), ('025.avi', 1), ('026.avi', 1), ('027.avi', 1), ('028.avi', 1), ('029.avi', 1), ('030.avi', 1), ('031.avi', 1), ('032.avi', 1), ('033.avi', 1), ('034.avi', 1), ('035.avi', 1), ('036.avi', 1), ('037.avi', 1), ('038.avi', 1), ('039.avi', 1), ('040.avi', 1), ('041.avi', 1), ('042.avi', 1), ('043.avi', 1), ('044.avi', 1), ('045.avi', 1), ('046.avi', 1), ('047.avi', 1), ('048.avi', 1), ('049.avi', 1), ('050.avi',

(8) Data_integration

In [40]:
N=80
num_test=20
num_train=num_audios-num_test
L=int(np.ceil(num_train/N)*num_slices)
i=0
for i in range(N):
    if i<79:
        print('Saving data part'+str(i+1)+'...')
        print('Saving test data')
        start=i*L
        end=(i+1)*L
        print(str(start)+' to '+str(end))
#         print(video_input[start:end,1,1,1,1])
#         print(audio_input[start:end,1,1])
        sio.savemat(integrate_data_path+'preprocessed_data_final_part'+str(i+1)+'.mat', mdict={'video_input': video_input[start:end,:,:,:,:], 'audio_input' : audio_input[start:end,:,:]})
    else:
        print('Saving data part'+str(i+1)+'...')
        start=i*L
        end=num_train*num_slices
        print(str(start)+' to '+str(end))
#         print(video_input[start:end,1,1,1,1])
#         print(audio_input[start:end,1,1])
        sio.savemat(integrate_data_path+'preprocessed_data_final_part'+str(i+1)+'.mat', mdict={'video_input': video_input[start:end,:,:,:,:], 'audio_input' : audio_input[start:end,:,:]})

print('Saving validation data...')
start=num_train*num_slices
print(str(start)+' to '+str(video_input.shape[0]))
# print(video_input[start:,1,1,1,1])
# print(audio_input[start:,1,1])
sio.savemat(integrate_data_path+'preprocessed_data_final_validation.mat', mdict={'video_input': video_input[start:,:,:,:,:], 'audio_input' : audio_input[start:,:,:]})

Saving data part1...
Saving test data
0 to 15
Saving data part2...
Saving test data
15 to 30
Saving data part3...
Saving test data
30 to 45
Saving data part4...
Saving test data
45 to 60
Saving data part5...
Saving test data
60 to 75
Saving data part6...
Saving test data
75 to 90
Saving data part7...
Saving test data
90 to 105
Saving data part8...
Saving test data
105 to 120
Saving data part9...
Saving test data
120 to 135
Saving data part10...
Saving test data
135 to 150
Saving data part11...
Saving test data
150 to 165
Saving data part12...
Saving test data
165 to 180
Saving data part13...
Saving test data
180 to 195
Saving data part14...
Saving test data
195 to 210
Saving data part15...
Saving test data
210 to 225
Saving data part16...
Saving test data
225 to 240
Saving data part17...
Saving test data
240 to 255
Saving data part18...
Saving test data
255 to 270
Saving data part19...
Saving test data
270 to 285
Saving data part20...
Saving test data
285 to 300
Saving data part21...
S